In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
!pip install torchvision

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        # normalize specific to inception model
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

# load CIFAR10 dataset
cifar10_train = CIFAR10(root='/mnt_mount/labproject_data', train=True, download=True, transform=transform)
cifar10_test = CIFAR10(root='/mnt_mount/labproject_data', train=False, download=True, transform=transform)


In [ ]:
dataloader_1 = torch.utils.data.DataLoader(cifar10_train, batch_size=100, shuffle=False, num_workers=1)
dataloader_2 = torch.utils.data.DataLoader(cifar10_test, batch_size=100, shuffle=False, num_workers=1)

In [ ]:
from torchvision.models import inception_v3
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np


# get embedding net
def get_embedding_net():
    model = inception_v3(pretrained=True)
    model.fc = torch.nn.Identity()  # replace the classifier with identity to get features
    model.eval()
    return model.to('cuda' if torch.cuda.is_available() else 'cpu')

# extract features
def extract_features(dataloader, model):
    features = []
    with torch.no_grad():
        for data, _ in dataloader:
            data = data.to('cuda' if torch.cuda.is_available() else 'cpu')
            features.append(model(data))
    return torch.cat(features).cpu().numpy()


embedding_net = get_embedding_net()

features1 = extract_features(dataloader_1, embedding_net)
features2 = extract_features(dataloader_2, embedding_net)


In [ ]:
features1.shape, features2.shape

In [ ]:
from labproject.metrics.sliced_wasserstein import sliced_wasserstein_distance

swd = sliced_wasserstein_distance(torch.from_numpy(features1)[:10000], torch.from_numpy(features2), num_projections=1000)